In [1]:
import sys
sys.path.append("C:/Users/user/meepc")
import numpy as np
import pandas as pd
from models import Hankel,Pipeline,Corrhankel
import warnings
warnings.simplefilter('ignore')
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
df_1 = pd.read_csv('~/data/ctown/dataset03.csv')
df_2 = pd.read_csv('~/data/ctown/dataset04.csv')
train_normal=pd.concat((df_1,df_2[df_2['ATT_FLAG']==0]),axis=0,ignore_index=True)
train_attack=df_2[df_2['ATT_FLAG']==1]

In [3]:
sensors=[col for col in train_normal.columns if col not in ['DATETIME','ATT_FLAG']]

In [4]:
scaler = StandardScaler()
X_normal=pd.DataFrame(data=scaler.fit_transform(train_normal.loc[:,sensors]),index=train_normal.index,columns=sensors)
X_attack = train_attack[sensors].reset_index().drop(columns=['index'])

In [5]:
hankel = Hankel()
corrhankel = Corrhankel()
lag = 60
stride = 0.5

In [6]:
corr_normal,nolag_normal = corrhankel.fit(X_normal.to_numpy(),lag,stride)

In [7]:
corr_attack,nolag_attack = corrhankel.fit(X_attack.to_numpy(),lag,stride)

In [8]:
df_test = pd.read_csv('~/data/ctown/test_dataset.csv')

In [9]:
test_combined = pd.concat((df_test,train_attack),axis=0)
X_test = pd.DataFrame(index=test_combined.index, columns=sensors, data=scaler.fit_transform(test_combined[sensors]))
corr_test,nolag_test = corrhankel.fit(X_test.to_numpy(),lag,stride)
Y_test = test_combined.loc[:,'ATT_FLAG']

In [10]:
labels = hankel.fit(np.array(Y_test),lag,stride)
y_actual = np.any(labels>0,axis=0).astype(int)

# Epasad with 1 cluster and no threshold tuning (training attack included in test data)

In [11]:
sensor_models= []
sensor_predicted = []
accuracy = []
precision = []
recall = []
fscore = []
for i,sens in enumerate(sensors):
    train_normal = X_normal.loc[:,sens].values
    train_attack = X_attack.loc[:,sens].values
    model = Pipeline()
    model.fit(train_normal,train_attack,lag,stride,optimal_k=1,tune=False,kscore_init='inertia',corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T)
    test = X_test.loc[:,sens].values
    y_predicted = model.predict(test,corr_test[:,i].reshape(nolag_test,len(X_test.columns)).T)
    sensor_predicted.append(y_predicted)
    accuracy.append(accuracy_score(y_actual,y_predicted))
    precision.append(precision_score(y_actual,y_predicted))
    recall.append(recall_score(y_actual,y_predicted)) 
    fscore.append(f1_score(y_actual,y_predicted))
    sensor_models.append(model)   

In [12]:
sensor_predicitions = np.asarray(sensor_predicted)
predicted_label=np.any(sensor_predicitions>0,axis=0).astype(int)
predicted_label

array([1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1])

In [13]:
print("Accuracy ",accuracy_score(y_actual,predicted_label))
print("Precision ",precision_score(y_actual,predicted_label))
print("Recall ",recall_score(y_actual,predicted_label))
print("F1-score ",f1_score(y_actual,predicted_label))

Accuracy  0.788235294117647
Precision  0.7692307692307693
Recall  0.8695652173913043
F1-score  0.8163265306122449


In [14]:
print("Accuracy -  Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(accuracy).mean(), np.median(np.asarray(accuracy)),np.asarray(accuracy).min(), np.asarray(accuracy).max()))
print("precision - Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(precision).mean(), np.median(np.asarray(precision)),np.asarray(precision).min(), np.asarray(precision).max()))
print("recall -    Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(recall).mean(), np.median(np.asarray(recall)),np.asarray(recall).min(), np.asarray(recall).max()))
print("f1 -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fscore).mean(), np.median(np.asarray(fscore)),np.asarray(fscore).min(), np.asarray(fscore).max()))

Accuracy -  Mean: 0.49931600547195637 Median : 0.5058823529411764 Min : 0.4588235294117647 Max : 0.6
precision - Mean: 0.6210436887933468 Median : 0.8571428571428571 Min : 0.0 Max : 1.0
recall -    Mean: 0.0839231547017189 Median : 0.08695652173913043 Min : 0.0 Max : 0.30434782608695654
f1 -        Mean: 0.14292442282802942 Median : 0.16 Min : 0.0 Max : 0.4516129032258065


# Multiple clusters: no threshold tuning (training included in test data)

In [15]:
fscores_k_1 = []
for k in range(2,8):
    sensor_models= []
    sensor_predicted = []
    accuracy = []
    precision = []
    recall = []
    fscore = []
    for i,sens in enumerate(sensors):
        train_normal = X_normal.loc[:,sens].values
        train_attack = X_attack.loc[:,sens].values
        model = Pipeline()
        model.fit(train_normal,train_attack,lag,stride,optimal_k = k,tune=False,kscore_init='inertia',corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T)
        test = X_test.loc[:,sens].values
        y_predicted = model.predict(test,corr_test=corr_test[:,i].reshape(nolag_test,len(X_test.columns)).T)
        sensor_predicted.append(y_predicted)
        accuracy.append(accuracy_score(y_actual,y_predicted))
        precision.append(precision_score(y_actual,y_predicted))
        recall.append(recall_score(y_actual,y_predicted)) 
        fscore.append(f1_score(y_actual,y_predicted))
        sensor_models.append(model)   
    sensor_predicted = np.asarray(sensor_predicted)
    y_predicted = np.any(sensor_predicted,axis=0).astype(int) 
    print("-------number of clusters------", k)
    print("Accuracy ",accuracy_score(y_actual,y_predicted))
    print("Precision ",precision_score(y_actual,y_predicted))
    print("Recall ",recall_score(y_actual,y_predicted))
    print("F1-score ",f1_score(y_actual,y_predicted))
    fscores_k_1.append(f1_score(y_actual,y_predicted))

-------number of clusters------ 2
Accuracy  0.7764705882352941
Precision  0.7872340425531915
Recall  0.8043478260869565
F1-score  0.795698924731183
-------number of clusters------ 3
Accuracy  0.7176470588235294
Precision  0.75
Recall  0.717391304347826
F1-score  0.7333333333333332
-------number of clusters------ 4
Accuracy  0.7294117647058823
Precision  0.7090909090909091
Recall  0.8478260869565217
F1-score  0.7722772277227722
-------number of clusters------ 5
Accuracy  0.6823529411764706
Precision  0.6610169491525424
Recall  0.8478260869565217
F1-score  0.7428571428571429
-------number of clusters------ 6
Accuracy  0.7176470588235294
Precision  0.6896551724137931
Recall  0.8695652173913043
F1-score  0.7692307692307693
-------number of clusters------ 7
Accuracy  0.6588235294117647
Precision  0.6231884057971014
Recall  0.9347826086956522
F1-score  0.7478260869565218


In [16]:
fscores_k_1

[0.795698924731183,
 0.7333333333333332,
 0.7722772277227722,
 0.7428571428571429,
 0.7692307692307693,
 0.7478260869565218]

# Epasad with 1 cluster : (training attack is not in test data)

In [17]:
X_test = pd.DataFrame(index=df_test.index, columns=sensors, data=scaler.fit_transform(df_test[sensors]))
Y_test = df_test.loc[:,'ATT_FLAG']
corr_test,nolag_test = corrhankel.fit(X_test.to_numpy(),lag,stride)

In [18]:
labels = hankel.fit(np.array(Y_test),lag,stride)
y_actual = np.any(labels>0,axis=0).astype(int)

In [19]:
sensor_models= []
sensor_predicted = []
accuracy = []
precision = []
recall = []
fscore = []
for i,sens in enumerate(sensors):
    train_normal = X_normal.loc[:,sens].values
    train_attack = X_attack.loc[:,sens].values
    model = Pipeline()
    model.fit(train_normal,train_attack,lag,stride,optimal_k=1,tune=False,kscore_init='inertia',corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T,corr_attack=corr_attack[:,i].reshape(nolag_attack,len(X_attack.columns)).T)
    test = X_test.loc[:,sens].values
    y_predicted = model.predict(test,corr_test[:,i].reshape(nolag_test,len(X_test.columns)).T)
    sensor_predicted.append(y_predicted)
    accuracy.append(accuracy_score(y_actual,y_predicted))
    precision.append(precision_score(y_actual,y_predicted))
    recall.append(recall_score(y_actual,y_predicted)) 
    fscore.append(f1_score(y_actual,y_predicted))
    sensor_models.append(model)   

In [20]:
sensor_predicitions = np.asarray(sensor_predicted)
predicted_label=np.any(sensor_predicitions>0,axis=0).astype(int)
predicted_label

array([1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 0])

In [21]:
print("Accuracy ",accuracy_score(y_actual,predicted_label))
print("Precision ",precision_score(y_actual,predicted_label))
print("Recall ",recall_score(y_actual,predicted_label))
print("F1-score ",f1_score(y_actual,predicted_label))

Accuracy  0.7205882352941176
Precision  0.625
Recall  0.8620689655172413
F1-score  0.7246376811594203


In [22]:
print("Accuracy -  Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(accuracy).mean(), np.median(np.asarray(accuracy)),np.asarray(accuracy).min(), np.asarray(accuracy).max()))
print("precision - Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(precision).mean(), np.median(np.asarray(precision)),np.asarray(precision).min(), np.asarray(precision).max()))
print("recall -    Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(recall).mean(), np.median(np.asarray(recall)),np.asarray(recall).min(), np.asarray(recall).max()))
print("f1 -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fscore).mean(), np.median(np.asarray(fscore)),np.asarray(fscore).min(), np.asarray(fscore).max()))

Accuracy -  Mean: 0.5906292749658002 Median : 0.5735294117647058 Min : 0.5588235294117647 Max : 0.6764705882352942
precision - Mean: 0.46396342675412433 Median : 0.5 Min : 0.0 Max : 1.0
recall -    Mean: 0.06655974338412189 Median : 0.034482758620689655 Min : 0.0 Max : 0.2413793103448276
f1 -        Mean: 0.11097061031053816 Median : 0.06666666666666667 Min : 0.0 Max : 0.3888888888888889


# Multiple clusters : No threshold tuning (training attack is not in test data)

In [23]:
fscores_k_2=[]
for k in range(2,8):
    sensor_models = []
    sensor_predicted = []
    accuracy = []
    precision = []
    recall = []
    fscore = []
    for i,sens in enumerate(sensors):
        train_normal = X_normal.loc[:,sens].values
        train_attack = X_attack.loc[:,sens].values
        model = Pipeline()
        model.fit(train_normal,train_attack,lag,stride,optimal_k=k,tune=False,kscore_init='inertia',corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T)
        test = X_test.loc[:,sens].values
        y_predicted = model.predict(test,corr_test=corr_test[:,i].reshape(nolag_test,len(X_normal.columns)).T)
        sensor_predicted.append(y_predicted)
        accuracy.append(accuracy_score(y_actual,y_predicted))
        precision.append(precision_score(y_actual,y_predicted))
        recall.append(recall_score(y_actual,y_predicted))
        fscore.append(f1_score(y_actual,y_predicted))
        sensor_models.append(model)    
    sensor_predicted = np.asarray(sensor_predicted)
    y_predicted = np.any(sensor_predicted,axis=0).astype(int)
    print("-------number of clusters------", k)
    print("Accuracy ",accuracy_score(y_actual,y_predicted))
    print("Precision ",precision_score(y_actual,y_predicted))
    print("Recall ",recall_score(y_actual,y_predicted))
    print("F1-score ",f1_score(y_actual,y_predicted))
    fscores_k_2.append(f1_score(y_actual,y_predicted))

-------number of clusters------ 2
Accuracy  0.6617647058823529
Precision  0.5833333333333334
Recall  0.7241379310344828
F1-score  0.6461538461538462
-------number of clusters------ 3
Accuracy  0.6617647058823529
Precision  0.5714285714285714
Recall  0.8275862068965517
F1-score  0.676056338028169
-------number of clusters------ 4
Accuracy  0.5882352941176471
Precision  0.5128205128205128
Recall  0.6896551724137931
F1-score  0.5882352941176471
-------number of clusters------ 5
Accuracy  0.6029411764705882
Precision  0.5208333333333334
Recall  0.8620689655172413
F1-score  0.6493506493506493
-------number of clusters------ 6
Accuracy  0.6029411764705882
Precision  0.5208333333333334
Recall  0.8620689655172413
F1-score  0.6493506493506493
-------number of clusters------ 7
Accuracy  0.5588235294117647
Precision  0.49056603773584906
Recall  0.896551724137931
F1-score  0.6341463414634146


In [24]:
fscores_k_2

[0.6461538461538462,
 0.676056338028169,
 0.5882352941176471,
 0.6493506493506493,
 0.6493506493506493,
 0.6341463414634146]

# Multiple clusters : threshold tuning (training attack is not in test data)

In [25]:
fscores_k_3 = []
for k in range(2,8):
    sensor_models= []
    sensor_predicted = []
    accuracy = []
    precision = []
    recall = []
    fscore = []
    for i,sens in enumerate(sensors):
        train_normal = X_normal.loc[:,sens].values
        train_attack = X_attack.loc[:,sens].values
        model = Pipeline()
        model.fit(train_normal,train_attack,lag,stride,optimal_k = k,tune=True,kscore_init='inertia',corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T,corr_attack=corr_attack[:,i].reshape(nolag_attack,len(X_attack.columns)).T)
        test = X_test.loc[:,sens].values
        y_predicted = model.predict(test,corr_test=corr_test[:,i].reshape(nolag_test,len(X_test.columns)).T)
        sensor_predicted.append(y_predicted)
        accuracy.append(accuracy_score(y_actual,y_predicted))
        precision.append(precision_score(y_actual,y_predicted))
        recall.append(recall_score(y_actual,y_predicted)) 
        fscore.append(f1_score(y_actual,y_predicted))
        sensor_models.append(model)   
    sensor_predicted = np.asarray(sensor_predicted)
    y_predicted = np.any(sensor_predicted,axis=0).astype(int)
    print("-------number of clusters------", k)
    print("Accuracy ",accuracy_score(y_actual,y_predicted))
    print("Precision ",precision_score(y_actual,y_predicted))
    print("Recall ",recall_score(y_actual,y_predicted))
    print("F1-score ",f1_score(y_actual,y_predicted))
    fscores_k_3.append(f1_score(y_actual,y_predicted))

-------number of clusters------ 2
Accuracy  0.4264705882352941
Precision  0.4264705882352941
Recall  1.0
F1-score  0.5979381443298969
-------number of clusters------ 3
Accuracy  0.4264705882352941
Precision  0.4264705882352941
Recall  1.0
F1-score  0.5979381443298969
-------number of clusters------ 4
Accuracy  0.45588235294117646
Precision  0.4393939393939394
Recall  1.0
F1-score  0.6105263157894737
-------number of clusters------ 5
Accuracy  0.5147058823529411
Precision  0.4642857142857143
Recall  0.896551724137931
F1-score  0.611764705882353
-------number of clusters------ 6
Accuracy  0.5147058823529411
Precision  0.4666666666666667
Recall  0.9655172413793104
F1-score  0.6292134831460675
-------number of clusters------ 7
Accuracy  0.4852941176470588
Precision  0.45161290322580644
Recall  0.9655172413793104
F1-score  0.6153846153846154


In [26]:
fscores_k_3

[0.5979381443298969,
 0.5979381443298969,
 0.6105263157894737,
 0.611764705882353,
 0.6292134831460675,
 0.6153846153846154]